In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=75ad272f4baf94b023bb8d822b816ad1f64f35cf32fcbfa1c9a735cedd221376
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, date_format, to_date


In [39]:
spark = SparkSession.builder \
    .appName("LAXPassengerAnalysis") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

In [40]:
file_path = "/content/lax_passengers_header.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [41]:
df.printSchema()


root
 |-- DataExtractDate: string (nullable = true)
 |-- ReportPeriod: string (nullable = true)
 |-- Terminal: string (nullable = true)
 |-- Arrival_Departure: string (nullable = true)
 |-- Domestic_International: string (nullable = true)
 |-- Passenger_Count: integer (nullable = true)



In [42]:
df.show(5)

+---------------+------------+-----------------+-----------------+----------------------+---------------+
|DataExtractDate|ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+---------------+------------+-----------------+-----------------+----------------------+---------------+
|  5/1/2014 0:00| 1/1/06 0:00|Imperial Terminal|          Arrival|              Domestic|            490|
|  5/1/2014 0:00| 1/1/06 0:00|Imperial Terminal|        Departure|              Domestic|            498|
|  5/1/2014 0:00| 1/1/06 0:00|   Misc. Terminal|          Arrival|              Domestic|            753|
|  5/1/2014 0:00| 1/1/06 0:00|   Misc. Terminal|        Departure|              Domestic|            688|
|  5/1/2014 0:00| 1/1/06 0:00|       Terminal 1|          Arrival|              Domestic|         401535|
+---------------+------------+-----------------+-----------------+----------------------+---------------+
only showing top 5 rows



In [43]:
df = df.withColumn("ReportPeriod", to_date(col("ReportPeriod"), "M/d/yyyy"))

In [44]:
df.select("ReportPeriod").show(5)

+------------+
|ReportPeriod|
+------------+
|  0006-01-01|
|  0006-01-01|
|  0006-01-01|
|  0006-01-01|
|  0006-01-01|
+------------+
only showing top 5 rows



In [45]:
df = df.filter(col("ReportPeriod").isNotNull())

In [46]:
filtered_df = df.filter(col("Terminal").isin(
    "Terminal 1", "Terminal 2", "Terminal 3",
    "Terminal 4", "Terminal 5",
    "Terminal 6", "Terminal 7",
    "Terminal 8", "Tom Bradley International Terminal"
))

In [47]:
filtered_df = filtered_df.withColumn("MonthYear", date_format(col("ReportPeriod"), "MM/yyyy"))

In [48]:
filtered_df.select("MonthYear").show(5)

+---------+
|MonthYear|
+---------+
|  01/0006|
|  01/0006|
|  01/0006|
|  01/0006|
|  01/0006|
+---------+
only showing top 5 rows



In [49]:
monthly_passengers = filtered_df.groupBy("MonthYear").agg(_sum("Passenger_Count").alias("TotalPassengers"))

In [50]:
busy_months = monthly_passengers.filter(col("TotalPassengers") > 5000000)

In [51]:
busy_months.show()

+---------+---------------+
|MonthYear|TotalPassengers|
+---------+---------------+
|  04/0016|        6109205|
|  06/0008|        5538956|
|  10/0016|        6629325|
|  04/0014|        5614336|
|  11/0015|        5838539|
|  12/0015|        6196206|
|  05/0011|        5476704|
|  06/0017|        7470964|
|  12/0007|        5186354|
|  06/0010|        5311142|
|  05/0017|        6933026|
|  08/0015|        6902789|
|  11/0014|        5288012|
|  05/0007|        5256763|
|  03/0014|        5622137|
|  05/0008|        5247792|
|  03/0008|        5232233|
|  12/0016|        6640915|
|  08/0014|        6595721|
|  03/0007|        5240144|
+---------+---------------+
only showing top 20 rows

